In [ ]:
import django_initialiser
from api.melody_utils.search import search_lcs, search_dtw
from api.melody_utils.extractor import melody_str_to_arr
import pandas as pd
import matplotlib.pyplot as plt
from api.models import Song

In [ ]:
input_search = [76, 75, 76, 75, 76, 71, 74, 72, 69, 57, 76, 75, 76, 75, 76, 71, 74, 72, 69, 57, 76, 75, 76, 75, 76, 71, 74, 72, 69, 57, 76, 75, 76, 75, 76, 71, 74, 72, 69, 57]
lune = Song.objects.filter(
    title__contains='lune'
)
lune_notes = melody_str_to_arr(lune[0].note_sequence)
print(lune_notes)

In [ ]:
def measure_varying_query():
    """Measure relation between query length and search time of all algorithms."""
    try:
        f = open('./results.csv', 'a')
        for i in range (3, 41):
            cur_search = input_search[:i]
            lcs = search_lcs(cur_search, return_time=True)
            fast_dtw = search_dtw(cur_search, 0, generate_graphs=False, fast=True, return_time=True)
            dtw = search_dtw(cur_search, 0, generate_graphs=False, fast=False, return_time=True)
            f.write(f'{i},{lcs},{fast_dtw},{dtw}\n')
            print(f'Len {i} done')
        f.close()

    except KeyboardInterrupt:
        f.close()
        pass

In [ ]:
def measure_similar_():
    """Measure relation between query length and LCS length"""
    try:
        f = open('./results_lcs.csv', 'a')
        f.write(f'i,lcs_%\n')
        for i in range (3, 41):
            cur_search = lune_notes[:i]
            lcs_results = search_lcs(cur_search)
            lcs_length = 0
            for res in lcs_results:
                lcs_length += res['lcs_length']
            
            lcs_avg = lcs_length / len(lcs_results)
            lcs_percentage = lcs_avg / i

            f.write(f'{i},{lcs_percentage}\n')
        f.close()

    except KeyboardInterrupt:
        f.close()
        pass

In [ ]:
df = pd.read_csv('results.csv')
print(df)
# plt.plot(pd['lcs_time'])
plt.plot(df['input_len'], df['lcs_time'], label='LCS')
plt.plot(df['input_len'], df['fast_dtw_time'], label='fastDTW')
plt.plot(df['input_len'], df['dtw_time'], label='DTW')
plt.ylabel("Search time (s)")
plt.xlabel("Query melody length")
plt.legend()
plt.savefig('./query_len.png')
plt.show()

In [ ]:
constant = 0

for i in range(0, 38):
    constant += df['dtw_time'][i] / df['lcs_time'][i]

constant = constant / len(df['lcs_time'])
print(constant)

In [ ]:
# res2 = measure_similar_()

df = pd.read_csv('results_lcs.csv')
print(df)
# plt.plot(pd['lcs_time'])
plt.plot(df['i'], df['lcs_%'], label='LCS')
plt.ylabel("Average LCS match rate")
plt.xlabel("Query melody length")
# plt.legend()
# plt.savefig('./query_len.png')
plt.show()